In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [87]:
input_path = '../data'
output_path = '../output'
dataname = f"{input_path}/south-building/processed_sparse.ply"
pcd = o3d.io.read_point_cloud(dataname)

In [88]:
front = [ -0.20840498240751903, -0.35440523720933897, -0.91157242781158865 ]
lookat = [ 0.14971626040172237, 0.67829902545490228, 1.6413716637270543 ]
up = [ 0.12429800846355013, -0.93407746196284847, 0.33473765868966493 ]
zoom = 0.21999999999999997

In [89]:
pcd.has_normals()

True

In [90]:
o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)

### Ball Pivoting Algorithm

In [99]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 7 * avg_dist

In [92]:
radius

0.06532734188509356

In [93]:
distances[:10] # distance for each point

DoubleVector[0.00520294, 0.0360043, 0.0639401, 0.00261841, 0.00924112, 0.0192439, 0.00311007, 0.012448, 0.000242819, 0.019581]

In [101]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius * 2]))

In [103]:
o3d.visualization.draw_geometries([bpa_mesh], zoom=zoom, front=front, lookat=lookat, up=up, window_name='BPA Mesh')

In [44]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

In [45]:
o3d.visualization.draw_geometries([dec_mesh], zoom=zoom, front=front, lookat=lookat, up=up, window_name='BPA Mesh')

#### Optional Post-Processing

In [46]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

TriangleMesh with 11261 points and 6382 triangles.

In [47]:
o3d.visualization.draw_geometries([dec_mesh], zoom=zoom, front=front, lookat=lookat, up=up, window_name='BPA Mesh Post Processed')

### Poisson Reconstruction
Depth and scale parameters

In [73]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=12, width=0, scale=1.1, linear_fit=False)[0]

In [74]:
o3d.visualization.draw_geometries([poisson_mesh], zoom=zoom, front=front, lookat=lookat, up=up, window_name='Poisson Mesh')

In [76]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

In [77]:
o3d.visualization.draw_geometries([p_mesh_crop], window_name='Poisson Mesh Cropped')

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D INFO]   -- Mouse view control --
[Open3D INFO]     Left button + drag         : Rotate.
[Open3D INFO]     Ctrl + left button + drag  : Translate.
[Open3D INFO]     Wheel button + drag        : Translate.
[Open3D INFO]     Shift + left button + drag : Roll.
[Open3D INFO]     Wheel                      : Zoom in/out.
[Open3D INFO] 
[Open3D INFO]   -- Keyboard view control --
[Open3D INFO]     [/]          : Increase/decrease field of view.
[Open3D INFO]     R            : Reset view point.
[Open3D INFO]     Ctrl/Cmd + C : Copy current view status into the clipboard.
[Open3D INFO]     Ctrl/Cmd + V : Paste view status from clipboard.
[Open3D INFO] 
[Open3D INFO]   -- General control --
[Open3D INFO]     Q, Esc       : Exit window.
[Open3D INFO]     H            : Print help message.
[Open3D INFO]     P, PrtScn    : Take a screen capture.
[Open3D INFO]     D            : Take a depth capture.
[

#### LoD 

In [81]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods = {}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(f'{path}/lod_{i}{extension}', mesh_lod)
        mesh_lods[i] = mesh_lod
    return mesh_lods

In [82]:
# large values: close look, small values: look from far away
my_lods = lod_mesh_export(bpa_mesh, [100000, 50000, 10000, 1000, 100], '.ply', output_path)

In [84]:
my_lods

{100000: TriangleMesh with 11721 points and 5840 triangles.,
 50000: TriangleMesh with 11721 points and 5840 triangles.,
 10000: TriangleMesh with 11721 points and 5840 triangles.,
 1000: TriangleMesh with 7962 points and 1000 triangles.,
 100: TriangleMesh with 7220 points and 100 triangles.}

In [86]:
o3d.visualization.draw_geometries([my_lods[100]], window_name='Poisson Mesh Cropped')